In [2]:
import pandas as pd
import numpy as np
import os
import nilearn
from nilearn import masking
from nilearn import plotting
from nilearn import image
import os
import fsl.wrappers as fsl
os.environ['FSLDIR'] = '/usr/local/fsl'
os.environ['FSLOUTPUTTYPE'] = 'NIFTI_GZ'

In [11]:
#df = pd.read_csv("Data_Analyzed_7Networks_23Dec2021.csv")
#regions = pd.read_csv("Desikan_Atlas/atlas-desikankilliany.csv")

In [34]:
#regions = pd.read_csv('Schaefer2018_400Parcels_17Networks_order_FSLMNI152_1mm.Centroid_RAS.csv')
#regions = pd.read_csv('atl-Buckner17.csv', delim_whitespace=True)
regions = pd.read_csv('BN_Atlas_246_LUT_text.csv')
regions

,Value,ROI_Name
0,211,mAmyg_L
1,212,mAmyg_R
2,213,lAmyg_L
3,214,lAmyg_R
4,215,rHipp_L
5,216,rHipp_R
6,217,cHipp_L
7,218,cHipp_R
8,219,vCa_L
9,220,vCa_R


In [28]:
regions.iloc[[1], 0]

1    2
Name: index, dtype: int64

## Creating Masks

In [35]:
for i, region in enumerate(regions['ROI_Name']):
    name = 'Brainnetome_masks/'+region+'.nii.gz'
    value = int(regions.iloc[[i], 0].values[0])
    fsl.fslmaths('BN_Atlas_246_2mm.nii.gz').thr(value-0.1).uthr(value+0.1).run(name)

## Add meants

In [59]:
af_proc = os.listdir('/media/mohammad/Mohammad/Academic/MSc/AD_SDB/fslvbm/')
#af_proc = glob.glob('/media/mohammad/Mohammad/Academic/MSc/AD_SDB/fslvbm/struc/*brain.nii.gz')
subs = []
for file in af_proc:
    if file.endswith('nii.gz'):
        subs.append(file)
subs.sort()

In [103]:
inds = list(df.index.drop_duplicates())

In [61]:
meants_dir = '/home/mohammad/Programs/Thesis3/vbm_sub_meants.txt'

lis = []
with open(meants_dir,'r') as meants:
   
    # reading each line    
    for line in meants:
        # reading each word        
        for meant in line.split():
            lis.append(float(meant))

In [62]:
atlas = pd.read_csv('/home/mohammad/DataScience/Thesis_AD/Thesis3/Doing_on_7Network_Atlas/Desikan_Atlas/atlas-desikankilliany.csv')
labels = atlas['label']

In [95]:
try:
    df.set_index("RID", inplace=True)
except:
    pass

errors = []
j=0
for j, sub in enumerate(subs):
    sub = sub.split('_')[-1]
    sub = sub[:-7]
    if int(sub) not in inds:
        print(sub)
        continue
    #subj = data.loc[[int(sub)]]
    for i, label in enumerate(labels):
        #print(label)
        reg = atlas.iloc[[i]]
        struc = reg["structure"].values[0]
        if struc.find("subcort")== -1:
            continue
        hem = reg["hemisphere"].values[0]
        if hem == "B":
            continue
        col_name = hem + 'H_' + label
        if col_name not in df:
            df.insert(df.shape[1], col_name, np.nan)
        df.loc[[int(sub)], col_name] = lis [ i + j*83]

5093
5109
5131


In [98]:
df["LH_amygdala"].isna().sum()

244

## Old Data Add Meants

In [99]:
pre_proc = os.listdir('/media/mohammad/Mohammad/Academic/MSc/Master_Thesis/3_fsl_VBM/fslvbm/Dataset')
#af_proc = glob.glob('/media/mohammad/Mohammad/Academic/MSc/AD_SDB/fslvbm/struc/*brain.nii.gz')
subs2 = []
for file in pre_proc:
    if file.endswith('nii.gz'):
        subs2.append(file)
subs2.sort()

In [100]:
meants_dir = '/home/mohammad/Programs/Thesis3/Previous_Analysis/vbm_sub_meants_old.txt'

lis = []
with open(meants_dir,'r') as meants:
   
    # reading each line    
    for line in meants:
        # reading each word        
        for meant in line.split():
            lis.append(float(meant))

In [106]:
for j, sub in enumerate(subs2):
    sub = sub.split('_')[-1]
    sub = sub[:-7]
    if int(sub) ==4943:
        continue
    #subj = data.loc[[int(sub)]]
    for i, label in enumerate(labels):
        reg = atlas.iloc[[i]]
        struc = reg["structure"].values[0]
        if struc.find("subcort")== -1:
            continue
        hem = reg["hemisphere"].values[0]
        if hem == "B":
            continue
        col_name = hem + 'H_' + label
        if col_name not in df:
            df.insert(df.shape[1], col_name, np.nan)
        df.loc[[int(sub)], col_name] = lis [ i + j*83]

In [108]:
df.loc[[4987]]

,Subject ID,Phase,Sex,Weight,Research Group,Study Date,Age,MMSCORE,MMDRAW,MMWRITE,...,LH_accumbensarea,LH_hippocampus,LH_amygdala,RH_thalamusproper,RH_caudate,RH_putamen,RH_pallidum,RH_accumbensarea,RH_hippocampus,RH_amygdala
RID,,,,,,,,,,,,,,,,,,,,,
4987,012_S_4987,NaN,1,98.0,MCI,NaN,69.4,25,1,2,...,0.448506,0.465066,0.667672,0.205614,0.327231,0.431497,0.034305,0.397746,0.43451,0.594866


In [109]:
df.to_csv("Data_with_SubCortical_24Dec2021.csv")